In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import astropy.coordinates as coord
from astropy.table import Table
from astropy.io import fits
import astropy.units as u

import eleanor

In [2]:
plt.style.use('notebook')

In [3]:
gid = np.load('../data/giants_ids.npz')['id']

In [5]:
N = np.size(gid)

In [6]:
def get_source(gid, smax=12):
    """
    Return a list of source objects found across the sectors
    gid -- gaia id of the source
    smax -- maximum sector id
    """
    sectors = []
    for i in range(smax):
        try:
            s_ = eleanor.Source(gaia=gid, sector=i+1)
            sectors += [s_]
        except (ValueError, UnicodeDecodeError, IndexError, RuntimeError, TypeError, NameError):
            pass
    
    return sectors

In [10]:
def get_fluxes(sectors, height=15, width=15, bkg_size=31, do_psf=False, do_pca=False):
    """
    Return a list of TargetData objects for each sector the source was observed
    sectors -- list of Source objects across the sectors
    height, width, bkg_size -- parameters for eleanor photometering
    do_psf, do_pca -- flags for additional eleanor analysis
    """
    data = []
    for s in sectors:
        try:
            d = eleanor.TargetData(s, height=height, width=width, bkg_size=bkg_size, do_psf=do_psf, do_pca=do_pca)
            data += [d]
        except (np.linalg.LinAlgError, ValueError, IndexError, RuntimeError, TypeError, NameError):
            pass
    
    return data

In [26]:
def extract_lightcurve(data):
    """
    Return a dictionary with extracted time, sector id, raw, corrected, and normalized fluxes
    data -- a list of TargetData objects across sectors
    """
    t = []
    fcorr = []
    fnorm = []
    fraw = []
    sector_id = []

    for e, d in enumerate(data):
        q = d.quality == 0
        
        t = np.concatenate([t, d.time[q]])
        fraw = np.concatenate([fcorr, d.raw_flux[q]])
        fcorr = np.concatenate([fcorr, d.corr_flux[q]])
        fnorm = np.concatenate([fnorm, d.corr_flux[q]/np.nanmedian(d.corr_flux[q])])
        
        sector_id = np.concatenate([sector_id, np.ones(np.sum(q))*e])

    outdict = {}
    outdict['t'] = t*u.day
    outdict['fraw'] = fraw
    outdict['fcorr'] = fcorr
    outdict['fnorm'] = fnorm
    outdict['sector_id'] = sector_id.astype('int')
    
    return outdict

In [35]:
e0 = 1
for e, sid in enumerate(gid[e0:10]):
    e += e0
    print(e, sid)
    
    # get source information
    sectors = get_source(sid, smax=21)
    
    # measure tess fluxes in all available sectors
    data = get_fluxes(sectors)
    
    # extract light curves
    outdict = extract_lightcurve(data)
    
    # save data
    tout = Table(outdict)
    tout.write('../data/bulk/lc_{:04d}.fits'.format(e), overwrite=True)


1 4793052149352862080
INFO: Found cached file /home/ana/.eleanor/mastDownload/HLSP/hlsp_eleanor_tess_ffi_postcard-s0004-4-1-cal-0510-0052_tess_v2_pc/hlsp_eleanor_tess_ffi_postcard-s0004-4-1-cal-0510-0052_tess_v2_bkg.fits with expected size 65286720. [astroquery.query]
INFO: Found cached file /home/ana/.eleanor/mastDownload/HLSP/hlsp_eleanor_tess_ffi_postcard-s0004-4-1-cal-0510-0052_tess_v2_pc/hlsp_eleanor_tess_ffi_postcard-s0004-4-1-cal-0510-0052_tess_v2_pc.fits with expected size 130665600. [astroquery.query]
INFO: Found cached file /home/ana/.eleanor/mastDownload/HLSP/hlsp_eleanor_tess_ffi_postcard-s0005-3-4-cal-0510-0484_tess_v2_pc/hlsp_eleanor_tess_ffi_postcard-s0005-3-4-cal-0510-0484_tess_v2_bkg.fits with expected size 73658880. [astroquery.query]
INFO: Found cached file /home/ana/.eleanor/mastDownload/HLSP/hlsp_eleanor_tess_ffi_postcard-s0005-3-4-cal-0510-0484_tess_v2_pc/hlsp_eleanor_tess_ffi_postcard-s0005-3-4-cal-0510-0484_tess_v2_pc.fits with expected size 147421440. [astroque

Set DATE-BEG to '2018-10-19T10:01:30.391' from MJD-BEG'. [astropy.wcs.wcs]


5


Set DATE-BEG to '2018-11-15T11:33:03.241' from MJD-BEG'. [astropy.wcs.wcs]


6


...

Set DATE-BEG to '2019-03-26T22:31:22.810' from MJD-BEG'. [astropy.wcs.wcs]


12
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]


Set DATE-BEG to '2019-05-21T11:01:47.002' from MJD-BEG'. [astropy.wcs.wcs]


13
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]


Set DATE-BEG to '2019-06-19T10:01:45.855' from MJD-BEG'. [astropy.wcs.wcs]
